Crea un programa que, llamando a una función clean_df(path), lea un archivo leíble por pandas, lo transforme a dataframe y lo limpie si es posible. 

La función clean_df() retornará el dataframe limpiado.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [8]:
def clean_df(path):
    # your code
    df = pd.read_csv(path)
    print(f"shape original: {df.shape}")
    # eliminar columnas Unnamed:
    for col in df.columns.tolist():
        if "Unnamed" in col:
            df.drop(col, 1, inplace = True)
    print()
    print(f"al eliminar columnas Unnamed: {df.shape}")
    # eliminar las columnas con más de 80% de nulos
    masde80 = df.columns[df.isnull().sum()/len(df)*100 > 80].tolist()
    if masde80:
        df.drop(masde80, 1, inplace = True)
    
    print()
    print(f"al eliminar columnas con más del 80% nulos: {df.shape}")
    # eliminar las filas con nulos
    df = df.dropna()
    
    print()
    print(f"al eliminar filas con algún nulo: {df.shape}")
    
    # pasar a numéricas las columnas
    object_col = df[df.dtypes[df.dtypes == "object"].index]
    
    if object_col.shape[1] != 0:
        for col in object_col.columns.tolist():
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])  
            
    print()
    print(f"al convertir columnas object a numéricas: {df.shape}")
    
    # eliminar outliers
    Q1, Q3 = df.quantile(0.25), df.quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    
    print()
    print(f"al eliminar outliers: {df.shape}")

    return df

En la función clean_df:

- path: representa la ubicación o url de donde pandas lee el archivo a cargar.

In [3]:
url = "https://raw.githubusercontent.com/Gabvaztor/data_science_jun_2020/master/week9/day2/data/student-mat.csv"
url2 = "https://raw.githubusercontent.com/Gabvaztor/data_science_jun_2020/master/week9/day2/data/US_Crime_Rates_1960_2014.csv"
url3 = "https://raw.githubusercontent.com/Geoyi/Cleaning-Titanic-Data/master/titanic_original.csv"

In [4]:
# dataframe original
df = pd.read_csv(url3)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   survived   1309 non-null   float64
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   float64
 6   parch      1309 non-null   float64
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(7), object(7)
memory usage: 143.4+ KB


In [9]:
# dataframe limpio 
df_limpio = clean_df(url3)
df_limpio.head()

shape original: (1310, 14)

al eliminar columnas Unnamed: (1310, 14)

al eliminar columnas con más del 80% nulos: (1310, 13)

al eliminar filas con algún nulo: (159, 13)

al convertir columnas object a numéricas: (159, 13)

al eliminar outliers: (123, 13)


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
1,1.0,1.0,2,1,0.9167,1.0,2.0,18,151.5500,41,2,2,52
5,1.0,1.0,3,1,48.0000,0.0,0.0,57,26.5500,88,2,8,55
6,1.0,1.0,4,0,63.0000,1.0,0.0,40,77.9583,85,2,1,33
8,1.0,1.0,5,0,53.0000,2.0,0.0,30,51.4792,32,2,20,5
12,1.0,1.0,7,0,24.0000,0.0,0.0,83,69.3000,14,0,16,63


Un artículo muy interesante: https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4